In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 17 10:40:55 2018

@author: chenz
"""
import numpy as np
import pandas as pd 
import torch
import torchvision
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.externals import joblib
import torch.utils.data as Data 

from getdata import getdata
from getdata_t import getdata_t

In [2]:
_, X,num_F,X_test,Y_test = getdata_t()
num_In = X.size()[1]*X.size()[2]
num_F = 81
###autoencoder##########################################################################
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        # encoder
        self.encoder = nn.Sequential(
            nn.Linear(num_In, num_F*2),
            nn.Tanh(),
            #nn.ReLU(),
            nn.Linear(num_F*2, num_F),
        )
        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(num_F, num_F*2),
            nn.Tanh(),
            #nn.ReLU(),
            nn.Linear(num_F*2, num_In),
            #nn.Sigmoid(), #0-1
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

autoencoder = AutoEncoder()

autoencoder = torch.load('./small/autoencoder.pkl')
print(autoencoder)

Loading ratings
   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
Loading movies
   movieId                    title  \
0        1         Toy Story (1995)   
1        2           Jumanji (1995)   
2        3  Grumpier Old Men (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
Loading tags
   userId  movieId                      tag   timestamp
0      15      339  sandra 'boring' bullock  1138537770
1      15     1955                  dentist  1193435061
2      15     7478                 Cambodia  1170560997
Loading genome_scores
   movieId  tagId  relevance
0        1      1    0.02500
1        1      2    0.02500
2        1      3    0.05775
Loading genome_tags
   tagId           tag
0      1           007
1      2  007 (seri

/Users/Lovely-white/anaconda/lib/python3.6/site-packages/torch/serialization.py:333: UserWarning: Couldn't retrieve source code for container of type AutoEncoder. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "
/Users/Lovely-white/anaconda/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/Lovely-white/anaconda/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool 

In [3]:

kmeans = joblib.load("./kmeans.m")
X_te = Variable(X_test.view(-1, num_In))
encoded_t, _ = autoencoder(X_te)
hf_t = encoded_t.data.numpy()
kmeans_t = kmeans.predict(hf_t)

/Users/Lovely-white/anaconda/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator KMeans from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
###Store movies for test users###################################################
movies = np.loadtxt("./small/mvrcmdlist.txt")
rcmdlist = []
for u in range(len(kmeans_t)):
    rcmd = movies[kmeans_t[u]-1][:50].copy()
    uwatched = list(np.where(X_test[u,:,0]==0))
    for mv in uwatched:
        if mv in rcmd: rcmd.remove(mv)
    rcmdlist.append(rcmd)

/Users/Lovely-white/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  


In [10]:
#f = open('prediction.txt', 'w')
#print recomend movies for test users and which movies they actually seen
path = 'ml-latest-small'
movie = pd.read_csv("./"+path+"/movie.csv")
movie = np.array(movie)
for i in range(len(rcmdlist)):
    print('====================================================================')
    print('For user ',i+1+X.shape[0])
    print('========Recommend Movies=======')
    for j in range(len(rcmdlist[i])):
        print('Movie name: ',movie[int(rcmdlist[i][j]),1],' ||genres: ',movie[int(rcmdlist[i][j]),2])
    print('========Actually Seen Movies=======')
    n = 0
    for j in range(len(Y_test[i+1+X.shape[0]])):
        #print(movie[int(Y_test[i+1+X.shape[0]][j][0]),1])
        if(int(Y_test[i+1+X.shape[0]][j][0]) < movie.shape[0]):
            print('Movie name: ',movie[int(Y_test[i+1+X.shape[0]][j][0]),1],' || genres: ',movie[int(Y_test[i+1+X.shape[0]][j][0]),2])
            #print('Rating: ',Y_test[i+1+X.shape[0]][j][1])
        if(int(Y_test[i+1+X.shape[0]][j][0]) in rcmdlist[i]):
            n+=1
    print('Same movies among 50 recommended:',n) 
#f.close()

For user  652
========Recommend Movies=======
Movie name:  Godfather, The (1972)  ||genres:  Crime|Drama
Movie name:  Shawshank Redemption, The (1994)  ||genres:  Crime|Drama
Movie name:  Godfather: Part II, The (1974)  ||genres:  Crime|Drama
Movie name:  American Beauty (1999)  ||genres:  Drama|Romance
Movie name:  Usual Suspects, The (1995)  ||genres:  Crime|Mystery|Thriller
Movie name:  Fargo (1996)  ||genres:  Comedy|Crime|Drama|Thriller
Movie name:  Silence of the Lambs, The (1991)  ||genres:  Crime|Horror|Thriller
Movie name:  Pulp Fiction (1994)  ||genres:  Comedy|Crime|Drama|Thriller
Movie name:  One Flew Over the Cuckoo's Nest (1975)  ||genres:  Drama
Movie name:  Schindler's List (1993)  ||genres:  Drama|War
Movie name:  Fight Club (1999)  ||genres:  Action|Crime|Drama|Thriller
Movie name:  Goodfellas (1990)  ||genres:  Crime|Drama
Movie name:  Shakespeare in Love (1998)  ||genres:  Comedy|Drama|Romance
Movie name:  Taxi Driver (1976)  ||genres:  Crime|Drama|Thriller
Movie na

Movie name:  2001: A Space Odyssey (1968)  ||genres:  Adventure|Drama|Sci-Fi
Movie name:  City of God (Cidade de Deus) (2002)  ||genres:  Action|Adventure|Crime|Drama|Thriller
Movie name:  Election (1999)  ||genres:  Comedy
Movie name:  Citizen Kane (1941)  ||genres:  Drama|Mystery
Movie name:  Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)  ||genres:  Drama|Film-Noir|Romance
Movie name:  Vertigo (1958)  ||genres:  Drama|Mystery|Romance|Thriller
Movie name:  Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)  ||genres:  Comedy|Romance
========Actually Seen Movies=======
Movie name:  Dirty Dancing (1987)  || genres:  Drama|Musical|Romance
Movie name:  Happy-Go-Lucky (2008)  || genres:  Comedy|Drama
Movie name:  No Nukes (1980)  || genres:  Documentary|Musical
Movie name:  Rock 'N' Roll High School (1979)  || genres:  Comedy|Musical
Movie name:  Walk the Line (2005)  || genres:  Drama|Musical|Romance
Movie name:  Robe, The (1953)  || genres:  Drama
Movie name:  Ugly, The (1997)  || genr

========Recommend Movies=======
Movie name:  Godfather, The (1972)  ||genres:  Crime|Drama
Movie name:  Shawshank Redemption, The (1994)  ||genres:  Crime|Drama
Movie name:  Godfather: Part II, The (1974)  ||genres:  Crime|Drama
Movie name:  American Beauty (1999)  ||genres:  Drama|Romance
Movie name:  Usual Suspects, The (1995)  ||genres:  Crime|Mystery|Thriller
Movie name:  Fargo (1996)  ||genres:  Comedy|Crime|Drama|Thriller
Movie name:  Silence of the Lambs, The (1991)  ||genres:  Crime|Horror|Thriller
Movie name:  Pulp Fiction (1994)  ||genres:  Comedy|Crime|Drama|Thriller
Movie name:  One Flew Over the Cuckoo's Nest (1975)  ||genres:  Drama
Movie name:  Schindler's List (1993)  ||genres:  Drama|War
Movie name:  Fight Club (1999)  ||genres:  Action|Crime|Drama|Thriller
Movie name:  Goodfellas (1990)  ||genres:  Crime|Drama
Movie name:  Shakespeare in Love (1998)  ||genres:  Comedy|Drama|Romance
Movie name:  Taxi Driver (1976)  ||genres:  Crime|Drama|Thriller
Movie name:  Sixth Sen

For user  667
========Recommend Movies=======
Movie name:  Godfather, The (1972)  ||genres:  Crime|Drama
Movie name:  Shawshank Redemption, The (1994)  ||genres:  Crime|Drama
Movie name:  Godfather: Part II, The (1974)  ||genres:  Crime|Drama
Movie name:  American Beauty (1999)  ||genres:  Drama|Romance
Movie name:  Usual Suspects, The (1995)  ||genres:  Crime|Mystery|Thriller
Movie name:  Fargo (1996)  ||genres:  Comedy|Crime|Drama|Thriller
Movie name:  Silence of the Lambs, The (1991)  ||genres:  Crime|Horror|Thriller
Movie name:  Pulp Fiction (1994)  ||genres:  Comedy|Crime|Drama|Thriller
Movie name:  One Flew Over the Cuckoo's Nest (1975)  ||genres:  Drama
Movie name:  Schindler's List (1993)  ||genres:  Drama|War
Movie name:  Fight Club (1999)  ||genres:  Action|Crime|Drama|Thriller
Movie name:  Goodfellas (1990)  ||genres:  Crime|Drama
Movie name:  Shakespeare in Love (1998)  ||genres:  Comedy|Drama|Romance
Movie name:  Taxi Driver (1976)  ||genres:  Crime|Drama|Thriller
Movie na

Movie name:  Pink Panther, The (2006)  || genres:  Adventure|Comedy|Crime
Movie name:  Born in East L.A. (1987)  || genres:  Comedy
Movie name:  Woodsman, The (2004)  || genres:  Drama
Movie name:  Spring Forward (1999)  || genres:  Drama
Movie name:  After School Special (a.k.a. Barely Legal) (2003)  || genres:  Comedy
Movie name:  Great Dictator, The (1940)  || genres:  Comedy|Drama|War
Movie name:  Substitute, The (1996)  || genres:  Action|Crime|Drama
Movie name:  Breakdown (1997)  || genres:  Action|Thriller
Movie name:  He Walked by Night (1948)  || genres:  Crime|Film-Noir|Thriller
Movie name:  Valiant (2005)  || genres:  Adventure|Animation|Children|Comedy|Fantasy|War
Movie name:  It Takes Two (1995)  || genres:  Children|Comedy
Movie name:  Butterfly Kiss (1995)  || genres:  Drama|Thriller
Same movies among 50 recommended: 0
